# Install Detectron2


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install pyyaml==5.1

from IPython.display import Image, clear_output

# Cerchiamo la versione che porti il match perfetto tra le librerie
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

clear_output()

# Import Libraries


In [4]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# Common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Import Model(s)
R101-FNP (Istance Segmentation)

In [5]:
ResNet_101_FPN = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
# Resnet_50_FPN = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml"

Specifichiamo il file di configurazione

In [6]:
cfg = get_cfg()

# Prendiamo le config dal file presente nel model zoo
cfg.merge_from_file(model_zoo.get_config_file(ResNet_101_FPN))

# Impostiamo una soglia sulle detection che andrà a scartare
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# Prendimao i pesi relativi ad un modello già addestrato su COCO 2017
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(ResNet_101_FPN)

# Bisogna aver attivato la GPU per eseguire 
predictor = DefaultPredictor(cfg)

model_final_a3ec72.pkl: 254MB [00:07, 34.4MB/s]                           


# Fine-Tuning

#Cityscapes

Importiamo script utili per Cityscapes

In [14]:
!python -m pip install cityscapesscripts

clear_output()

Implementiamo le funzioni di cityscapes.py un pelo modificate

In [15]:
import functools
import json
import logging
import multiprocessing as mp
import numpy as np
import os
from itertools import chain
import pycocotools.mask as mask_util
from PIL import Image

from detectron2.structures import BoxMode
from detectron2.utils.comm import get_world_size
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger

from cityscapesscripts.helpers.labels import labels
from cityscapesscripts.helpers.labels import id2label, name2label

try:
    import cv2
except ImportError:
    # OpenCV is an optional dependency at the moment
    pass


logger = logging.getLogger(__name__)


def _get_cityscapes_files(image_dir, gt_dir):
    files = []
    # scan through the directory
    cities = PathManager.ls(image_dir)
    logger.info(f"{len(cities)} cities found in '{image_dir}'.")
    for city in cities:
        city_img_dir = os.path.join(image_dir, city)
        city_gt_dir = os.path.join(gt_dir, city)
        for basename in PathManager.ls(city_img_dir):
            image_file = os.path.join(city_img_dir, basename)

            suffix = "leftImg8bit.png"
            assert basename.endswith(suffix), basename
            basename = basename[: -len(suffix)]

            instance_file = os.path.join(city_gt_dir, basename + "gtFine_instanceIds.png")
            label_file = os.path.join(city_gt_dir, basename + "gtFine_labelIds.png")
            json_file = os.path.join(city_gt_dir, basename + "gtFine_polygons.json")

            files.append((image_file, instance_file, label_file, json_file))
    assert len(files), "No images found in {}".format(image_dir)
    for f in files[0]:
        assert PathManager.isfile(f), f
    return files


def load_cityscapes_instances(image_dir, gt_dir, from_json=True, to_polygons=True):
    """
    Args:
        image_dir (str): path to the raw dataset. e.g., "~/cityscapes/leftImg8bit/train".
        gt_dir (str): path to the raw annotations. e.g., "~/cityscapes/gtFine/train".
        from_json (bool): whether to read annotations from the raw json file or the png files.
        to_polygons (bool): whether to represent the segmentation as polygons
            (COCO's format) instead of masks (cityscapes's format).

    Returns:
        list[dict]: a list of dicts in Detectron2 standard format. (See
        `Using Custom Datasets </tutorials/datasets.html>`_ )
    """
    if from_json:
        assert to_polygons, (
            "Cityscapes's json annotations are in polygon format. "
            "Converting to mask format is not supported now."
        )
    files = _get_cityscapes_files(image_dir, gt_dir)

    logger.info("Preprocessing cityscapes annotations ...")
    # This is still not fast: all workers will execute duplicate works and will
    # take up to 10m on a 8GPU server.
    pool = mp.Pool(processes=max(mp.cpu_count() // get_world_size() // 2, 4))

    ret = pool.map(
        functools.partial(_cityscapes_files_to_dict, from_json=from_json, to_polygons=to_polygons),
        files,
    )
    logger.info("Loaded {} images from {}".format(len(ret), image_dir))

    # Map cityscape ids to contiguous ids
    from cityscapesscripts.helpers.labels import labels

    labels = [l for l in labels if l.hasInstances and not l.ignoreInEval]
    dataset_id_to_contiguous_id = {l.id: idx for idx, l in enumerate(labels)}
    for dict_per_image in ret:
        for anno in dict_per_image["annotations"]:
            anno["category_id"] = dataset_id_to_contiguous_id[anno["category_id"]]
    return ret


def load_cityscapes_semantic(image_dir, gt_dir):
    """
    Args:
        image_dir (str): path to the raw dataset. e.g., "~/cityscapes/leftImg8bit/train".
        gt_dir (str): path to the raw annotations. e.g., "~/cityscapes/gtFine/train".

    Returns:
        list[dict]: a list of dict, each has "file_name" and
            "sem_seg_file_name".
    """
    ret = []
    # gt_dir is small and contain many small files. make sense to fetch to local first
    gt_dir = PathManager.get_local_path(gt_dir)
    for image_file, _, label_file, json_file in _get_cityscapes_files(image_dir, gt_dir):
        label_file = label_file.replace("labelIds", "labelTrainIds")

        with PathManager.open(json_file, "r") as f:
            jsonobj = json.load(f)
        ret.append(
            {
                "file_name": image_file,
                "sem_seg_file_name": label_file,
                "height": jsonobj["imgHeight"],
                "width": jsonobj["imgWidth"],
            }
        )
    assert len(ret), f"No images found in {image_dir}!"
    assert PathManager.isfile(
        ret[0]["sem_seg_file_name"]
    ), "Please generate labelTrainIds.png with cityscapesscripts/preparation/createTrainIdLabelImgs.py"  # noqa
    return ret


def _cityscapes_files_to_dict(files, from_json, to_polygons):
    """
    Parse cityscapes annotation files to a instance segmentation dataset dict.

    Args:
        files (tuple): consists of (image_file, instance_id_file, label_id_file, json_file)
        from_json (bool): whether to read annotations from the raw json file or the png files.
        to_polygons (bool): whether to represent the segmentation as polygons
            (COCO's format) instead of masks (cityscapes's format).

    Returns:
        A dict in Detectron2 Dataset format.
    """
    from cityscapesscripts.helpers.labels import id2label, name2label

    image_file, instance_id_file, _, json_file = files

    annos = []

    if from_json:
        from shapely.geometry import MultiPolygon, Polygon

        with PathManager.open(json_file, "r") as f:
            jsonobj = json.load(f)
        ret = {
            "file_name": image_file,
            "image_id": os.path.basename(image_file),
            "height": jsonobj["imgHeight"],
            "width": jsonobj["imgWidth"],
        }

        # `polygons_union` contains the union of all valid polygons.
        polygons_union = Polygon()

        # CityscapesScripts draw the polygons in sequential order
        # and each polygon *overwrites* existing ones. See
        # (https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/preparation/json2instanceImg.py) # noqa
        # We use reverse order, and each polygon *avoids* early ones.
        # This will resolve the ploygon overlaps in the same way as CityscapesScripts.
        for obj in jsonobj["objects"][::-1]:
            if "deleted" in obj:  # cityscapes data format specific
                continue
            label_name = obj["label"]

            try:
                label = name2label[label_name]
            except KeyError:
                if label_name.endswith("group"):  # crowd area
                    label = name2label[label_name[: -len("group")]]
                else:
                    raise
            if label.id < 0:  # cityscapes data format
                continue

            # Cityscapes's raw annotations uses integer coordinates
            # Therefore +0.5 here
            poly_coord = np.asarray(obj["polygon"], dtype="f4") + 0.5
            # CityscapesScript uses PIL.ImageDraw.polygon to rasterize
            # polygons for evaluation. This function operates in integer space
            # and draws each pixel whose center falls into the polygon.
            # Therefore it draws a polygon which is 0.5 "fatter" in expectation.
            # We therefore dilate the input polygon by 0.5 as our input.
            poly = Polygon(poly_coord).buffer(0.5, resolution=4)

            if not label.hasInstances or label.ignoreInEval:
                # even if we won't store the polygon it still contributes to overlaps resolution
                polygons_union = polygons_union.union(poly)
                continue

            # Take non-overlapping part of the polygon
            poly_wo_overlaps = poly.difference(polygons_union)
            if poly_wo_overlaps.is_empty:
                continue
            polygons_union = polygons_union.union(poly)

            anno = {}
            anno["iscrowd"] = label_name.endswith("group")
            anno["category_id"] = label.id

            if isinstance(poly_wo_overlaps, Polygon):
                poly_list = [poly_wo_overlaps]
            elif isinstance(poly_wo_overlaps, MultiPolygon):
                poly_list = poly_wo_overlaps.geoms
            else:
                raise NotImplementedError("Unknown geometric structure {}".format(poly_wo_overlaps))

            poly_coord = []
            for poly_el in poly_list:
                # COCO API can work only with exterior boundaries now, hence we store only them.
                # TODO: store both exterior and interior boundaries once other parts of the
                # codebase support holes in polygons.
                poly_coord.append(list(chain(*poly_el.exterior.coords)))
            anno["segmentation"] = poly_coord
            (xmin, ymin, xmax, ymax) = poly_wo_overlaps.bounds

            anno["bbox"] = (xmin, ymin, xmax, ymax)
            anno["bbox_mode"] = BoxMode.XYXY_ABS

            annos.append(anno)
    else:
        # See also the official annotation parsing scripts at
        # https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/evaluation/instances2dict.py  # noqa
        with PathManager.open(instance_id_file, "rb") as f:
            inst_image = np.asarray(Image.open(f), order="F")
        # ids < 24 are stuff labels (filtering them first is about 5% faster)
        flattened_ids = np.unique(inst_image[inst_image >= 24])

        ret = {
            "file_name": image_file,
            "image_id": os.path.basename(image_file),
            "height": inst_image.shape[0],
            "width": inst_image.shape[1],
        }

        for instance_id in flattened_ids:
            # For non-crowd annotations, instance_id // 1000 is the label_id
            # Crowd annotations have <1000 instance ids
            label_id = instance_id // 1000 if instance_id >= 1000 else instance_id
            label = id2label[label_id]
            if not label.hasInstances or label.ignoreInEval:
                continue

            anno = {}
            anno["iscrowd"] = instance_id < 1000
            anno["category_id"] = label.id

            mask = np.asarray(inst_image == instance_id, dtype=np.uint8, order="F")

            inds = np.nonzero(mask)
            ymin, ymax = inds[0].min(), inds[0].max()
            xmin, xmax = inds[1].min(), inds[1].max()
            anno["bbox"] = (xmin, ymin, xmax, ymax)
            if xmax <= xmin or ymax <= ymin:
                continue
            anno["bbox_mode"] = BoxMode.XYXY_ABS
            if to_polygons:
                # This conversion comes from D4809743 and D5171122,
                # when Mask-RCNN was first developed.
                contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[
                    -2
                ]
                polygons = [c.reshape(-1).tolist() for c in contours if len(c) >= 3]
                # opencv's can produce invalid polygons
                if len(polygons) == 0:
                    continue
                anno["segmentation"] = polygons
            else:
                anno["segmentation"] = mask_util.encode(mask[:, :, None])[0]
            annos.append(anno)
    ret["annotations"] = annos
    return ret



ModuleNotFoundError: ignored

Registriamo i dataset per cityscapes

In [ ]:
cityscapes_classes = [k.name for k in labels if k.hasInstances and not k.ignoreInEval]

image_dir = "/content/drive/MyDrive/datasets/cityscapes/leftImg8bit/"
gt_dir = "/content/drive/MyDrive/datasets/cityscapes/gtFine/"

for d in ["train"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")

for d in ["val"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")

for d in ["test"]:
  DatasetCatalog.register("cityscapes_" + d, lambda x=image_dir+d, y=gt_dir+d: load_cityscapes_instances(x, y))
  MetadataCatalog.get("cityscapes_" + d).set(thing_classes=cityscapes_classes, evaluator_type="coco")


Fine-tuning vero e proprio !

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(ResNet_101_FPN))
cfg.DATASETS.TRAIN = ("cityscapes_train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(ResNet_101_FPN)

# default 512
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

# default 16
cfg.SOLVER.IMS_PER_BATCH = 2

# default 40.000
cfg.SOLVER.MAX_ITER = 200
cfg.SOLVER.STEPS = (60,)

# Classi di Cityscapes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Print training metrics

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

#Save Models & Evaluate

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

COCO Evaluator API per compiere test e stampare AP

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("cityscapes_val", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "cityscapes_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

WARNING [01/28 15:33:43 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[01/28 15:34:34 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/28 15:34:34 d2.data.common]: Serializing 500 elements to byte tensors and concatenating them all ...
[01/28 15:34:34 d2.data.common]: Serialized dataset takes 12.86 MiB
[01/28 15:34:34 d2.evaluation.evaluator]: Start inference on 500 batches


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[01/28 15:34:37 d2.evaluation.evaluator]: Inference done 11/500. Dataloading: 0.0024 s/iter. Inference: 0.1528 s/iter. Eval: 0.0580 s/iter. Total: 0.2133 s/iter. ETA=0:01:44
[01/28 15:34:42 d2.evaluation.evaluator]: Inference done 34/500. Dataloading: 0.0027 s/iter. Inference: 0.1476 s/iter. Eval: 0.0690 s/iter. Total: 0.2196 s/iter. ETA=0:01:42
[01/28 15:34:48 d2.evaluation.evaluator]: Inference done 57/500. Dataloading: 0.0029 s/iter. Inference: 0.1491 s/iter. Eval: 0.0688 s/iter. Total: 0.2210 s/iter. ETA=0:01:37
[01/28 15:34:53 d2.evaluation.evaluator]: Inference done 76/500. Dataloading: 0.0030 s/iter. Inference: 0.1499 s/iter. Eval: 0.0805 s/iter. Total: 0.2336 s/iter. ETA=0:01:39
[01/28 15:34:58 d2.evaluation.evaluator]: Inference done 94/500. Dataloading: 0.0030 s/iter. Inference: 0.1500 s/iter. Eval: 0.0905 s/iter. Total: 0.2437 s/iter. ETA=0:01:38
[01/28 15:35:03 d2.evaluation.evaluator]: Inference done 114/500. Dataloading: 0.0030 s/iter. Inference: 0.1500 s/iter. Eval: 0.09

# WildDash

In [7]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_train", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_train.json", "/content/drive/MyDrive/datasets/wd_public_02/images_train")


from detectron2.data.datasets import register_coco_instances
register_coco_instances("dash_test", {}, "/content/drive/MyDrive/datasets/wd_public_02/wilddash_test.json", "/content/drive/MyDrive/datasets/wd_public_02/images_test")

In [19]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dash_train",)
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
# Classi di WildDash
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[01/29 11:05:53 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (14, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (14,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (52, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (52,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (13, 256, 1, 

[01/29 11:05:54 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual fl

[01/29 11:06:12 d2.utils.events]:  eta: 0:06:26  iter: 19  total_loss: 4.369  loss_cls: 2.557  loss_box_reg: 0.8576  loss_mask: 0.692  loss_rpn_cls: 0.05647  loss_rpn_loc: 0.1638  time: 0.8556  data_time: 0.1256  lr: 9.7405e-06  max_mem: 3947M
[01/29 11:06:28 d2.utils.events]:  eta: 0:06:07  iter: 39  total_loss: 4.109  loss_cls: 2.353  loss_box_reg: 0.8219  loss_mask: 0.6872  loss_rpn_cls: 0.05066  loss_rpn_loc: 0.1116  time: 0.8198  data_time: 0.0206  lr: 1.9731e-05  max_mem: 3947M
[01/29 11:06:44 d2.utils.events]:  eta: 0:05:51  iter: 59  total_loss: 3.619  loss_cls: 1.946  loss_box_reg: 0.8404  loss_mask: 0.6714  loss_rpn_cls: 0.0432  loss_rpn_loc: 0.08607  time: 0.8092  data_time: 0.0383  lr: 2.972e-05  max_mem: 3947M
[01/29 11:07:00 d2.utils.events]:  eta: 0:05:35  iter: 79  total_loss: 3.041  loss_cls: 1.352  loss_box_reg: 0.8247  loss_mask: 0.6522  loss_rpn_cls: 0.03976  loss_rpn_loc: 0.1336  time: 0.8076  data_time: 0.0262  lr: 3.9711e-05  max_mem: 3947M
[01/29 11:07:17 d2.uti

#Save Models & Evaluate

In [20]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [21]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("dash_test", cfg, False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dash_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

WARNING [01/29 11:12:44 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [01/29 11:12:44 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/29 11:12:44 d2.data.datasets.coco]: Loaded 890 images in COCO format from /content/drive/MyDrive/datasets/wd_public_02/wilddash_test.json
[01/29 11:12:44 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/29 11:12:44 d2.data.common]: Serializing 890 elements to byte tensors and concatenating them all ...
[01/29 11:12:44 d2.data.common]: Serialized dataset takes 5.77 MiB
[01/29 11:12:44 d2.evaluation.evaluator]: Start inference on 890 batches


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[01/29 11:12:47 d2.evaluation.evaluator]: Inference done 11/890. Dataloading: 0.0015 s/iter. Inference: 0.1578 s/iter. Eval: 0.0124 s/iter. Total: 0.1717 s/iter. ETA=0:02:30
[01/29 11:12:52 d2.evaluation.evaluator]: Inference done 39/890. Dataloading: 0.0019 s/iter. Inference: 0.1576 s/iter. Eval: 0.0201 s/iter. Total: 0.1797 s/iter. ETA=0:02:32
[01/29 11:12:57 d2.evaluation.evaluator]: Inference done 67/890. Dataloading: 0.0020 s/iter. Inference: 0.1578 s/iter. Eval: 0.0202 s/iter. Total: 0.1801 s/iter. ETA=0:02:28
[01/29 11:13:02 d2.evaluation.evaluator]: Inference done 94/890. Dataloading: 0.0020 s/iter. Inference: 0.1578 s/iter. Eval: 0.0218 s/iter. Total: 0.1817 s/iter. ETA=0:02:24
[01/29 11:13:07 d2.evaluation.evaluator]: Inference done 122/890. Dataloading: 0.0020 s/iter. Inference: 0.1576 s/iter. Eval: 0.0212 s/iter. Total: 0.1810 s/iter. ETA=0:02:18
[01/29 11:13:12 d2.evaluation.evaluator]: Inference done 151/890. Dataloading: 0.0020 s/iter. Inference: 0.1575 s/iter. Eval: 0.0

In [ ]:
data: List[Dict] = DatasetCatalog.get("wildash_test")

In [ ]:
MetadataCatalog.get("my_dataset").thing_classes

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output